In [1]:
import geopandas as gpd
import pandas as pd
from tqdm import tqdm

tqdm.pandas()

### Kontur population database loading

Caution: runnning this could be long!

In [8]:
### Chunking, last element determinet by hand dichotomy :)
chunks = []
for i in range(0, 32000000, 1000000):
    chunk_start = i
    chunk_end = i + 1000000
    chunks.append(slice(chunk_start, chunk_end))

chunks

[slice(0, 1000000, None),
 slice(1000000, 2000000, None),
 slice(2000000, 3000000, None),
 slice(3000000, 4000000, None),
 slice(4000000, 5000000, None),
 slice(5000000, 6000000, None),
 slice(6000000, 7000000, None),
 slice(7000000, 8000000, None),
 slice(8000000, 9000000, None),
 slice(9000000, 10000000, None),
 slice(10000000, 11000000, None),
 slice(11000000, 12000000, None),
 slice(12000000, 13000000, None),
 slice(13000000, 14000000, None),
 slice(14000000, 15000000, None),
 slice(15000000, 16000000, None),
 slice(16000000, 17000000, None),
 slice(17000000, 18000000, None),
 slice(18000000, 19000000, None),
 slice(19000000, 20000000, None),
 slice(20000000, 21000000, None),
 slice(21000000, 22000000, None),
 slice(22000000, 23000000, None),
 slice(23000000, 24000000, None),
 slice(24000000, 25000000, None),
 slice(25000000, 26000000, None),
 slice(26000000, 27000000, None),
 slice(27000000, 28000000, None),
 slice(28000000, 29000000, None),
 slice(29000000, 30000000, None),
 slic

In [16]:
gpd_list = []
for chunk in chunks:
    kontour = gpd.read_file(
        "data/kontur_population.gpkg", ignore_geometry=True, rows=chunk
    )
    gpd_list.append(kontour)

In [20]:
# forgot the last one..
last_one = gpd.read_file(
    "data/kontur_population.gpkg", ignore_geometry=True, rows=slice(32000000, 33000000)
)

In [22]:
last_one

,h3,population
0,88099c912dfffff,5.0
1,88099c912bfffff,1.0
2,88099c9129fffff,2.0
3,88099c911bfffff,1.0
4,88099c9119fffff,1.0
...,...,...
589548,8800189467fffff,8.0
589549,8800189465fffff,3.0
589550,880018942dfffff,1.0
589551,8800189425fffff,2.0


In [23]:
giga_gpd = pd.concat([giga_gpd, last_one])

In [25]:
giga_gpd.to_csv("data/kontour_400.csv")

In [ ]:
## OLD

In [1]:
# giga_gpd.info(memory_usage="deep")

In [2]:
# kontour_r6=pd.read_csv('kontour_400.csv')

Setting h3 index as df dataframe (it's an 'index' so better use it!). It should improve query time (we have 32 Million lines!) 

In [4]:
# kontour_neo=kontour_r6.drop(columns='Unnamed: 0').set_index('h3')

In [5]:
# hex_ids=["88f2a40257fffff","880018942dfffff","8800189467fffff",'881973d603fffff', '8819739a0dfffff', '8819710963fffff', '8819714e4dfffff', '881955cec3fffff', '8819468cc7fffff', '881955965dfffff', '881973b4ddfffff', '8819716f1bfffff', '8819735005fffff', '8819714c45fffff', '8819551829fffff', '881973af5bfffff', '88197382ebfffff', '8819464ab5fffff', '8819469b39fffff', '88194691b7fffff', '88197142b5fffff', '881946194bfffff', '8819468ec5fffff', '8819737061fffff', '881973793bfffff', '8819714693fffff', '8819559411fffff', '8819717131fffff', '881973d657fffff', '8819717069fffff', '881946d5e1fffff', '8819716663fffff', '8819733a39fffff', '881946c5e7fffff', '8819734235fffff', '8819460de1fffff', '88197154d7fffff', '8819467a35fffff', '881954a5bbfffff', '8819706ca1fffff', '8819468d11fffff', '8819736cd9fffff', '8819714a85fffff', '881973a0c7fffff', '881973246dfffff', '88197148e3fffff', '881973b6b9fffff', '881955d32dfffff', '8819730605fffff']

In [6]:
# %timeit kontour_r6.query('h3 ==  @hex_ids')['population'].values

In [ ]:
# %timeit kontour_neo.loc[kontour_neo.index.intersection(hex_ids),'population'].values

361 µs ± 1.79 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


Much faster!

### Test 

In [8]:
# data_airport=pd.read_csv('Data/pb_arpt_v2.csv')
# data_airport

In [9]:
# def find_population_for_radius(hex_id, R):
#     # finding the hexagon edge size, based on the index and the area of an hexagon A=3.sqrt(3)/2.a²
#     cell_edge=np.sqrt(2/(3*np.sqrt(3))*h3.cell_area(hex_id))
#     # Getting the largest k_ring fitting in the circle of radius R. K_ring is an hexagon, whose large radius is equals to (2n+1) the small radius of the base hexagon.
#     # Small radius of the base hexagon is equal to sqrt(3)/2.big radius which is equal to edge length, hence the formula below
#     n = int(np.floor((R)/(np.sqrt(3)*cell_edge)-1/2))
#     # Getting the ids of all the hexagons inside the k_ring
#     hex_ids = list(h3.k_ring(hex_id, n))
#     # getting their surfaces for surface normalisation
#     surfaces = sum([h3.cell_area(id) for id in hex_ids])

#     # Getting the population inside each hexagon
#     # ID vector query for much fatser processing when looking at many neigbors
#     pop= sum(kontour_neo.loc[kontour_neo.index.intersection(hex_ids),'population'].values)

#     # Normalisation: not all hexes are the same size. Meaning n won't be the same for different regions.
#     # Using a very high resolution grid allows this to have little effect on the surface considered:
#     # When small hexagons are very small against the 'big' hexagon of big radius R, there will always be an hex ring close to it.
#     # Still, a normalisation is made to have the population adjusted for the R hex surface for all the places investigated.

#     pop_norm = pop/surfaces*3*np.sqrt(3)/2*R**2
#     return pop_norm


# # Apply the find_k_nearest_hexagons and find_population_for_hexagons functions to each row of the point_data DataFrame
# data_airport['hex_id'] = data_airport.apply(lambda row: h3.geo_to_h3(row['lat'], row['lon'], 8), axis=1)


# data_airport.progress_apply(lambda row: find_population_for_radius(row['hex_id'], 50), axis=1)


# data_airport['surrounding_population_level_1'] = data_airport.progress_apply(lambda row: find_population_for_hexagons(row['hex_id'], 1), axis=1)
# data_airport['surrounding_population_level_2'] = data_airport.progress_apply(lambda row: find_population_for_hexagons(row['hex_id'], 2), axis=1)
# data_airport['surrounding_population_level_3'] = data_airport.progress_apply(lambda row: find_population_for_hexagons(row['hex_id'], 3), axis=1)
# data_airport['surrounding_population_level_4'] = data_airport.progress_apply(lambda row: find_population_for_hexagons(row['hex_id'], 8), axis=1)